---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    algorithm_list = []
    
    for G in P1_Graphs:
        cluster = nx.average_clustering(G)
        short_path = nx.average_shortest_path_length(G)
        
        # Create histogram
        degrees = G.degree()
        degree_vals = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_vals]
        
        if len(histogram) > 10:
            algorithm_list.append('PA')
        elif cluster < 0.1:
            algorithm_list.append('SW_H')
        else:
            algorithm_list.append('SW_L')
        
    return algorithm_list

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [5]:
G.nodes(data =  True)

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0}),
 (10, {'Department': 9, 'ManagementSalary': 0.0}),
 (11, {'Department': 14, 'ManagementSalary': 0.0}),
 (12, {'Department': 14, 'ManagementSalary': 1.0}),
 (13, {'Department': 26, 'ManagementSalary': 1.0}),
 (14, {'Department': 4, 'ManagementSalary': nan}),
 (15, {'Department': 17, 'ManagementSalary': 0.0}),
 (16, {'Department': 34, 'ManagementSalary': 0.0}),
 (17, {'Department': 1, 'ManagementSalary': 0.0}),
 (18, {'Department': 1, 'ManagementSalary': nan}),
 (19, {'Department': 14, 'Ma

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

# Define features
df = pd.DataFrame(index = G.nodes())
df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
df['Degree'] = pd.Series(nx.degree(G))
df['Betweennes_Centrality'] = pd.Series(nx.betweenness_centrality(G))
df['Degree_Centrality'] = pd.Series(nx.degree_centrality(G))
df['Pagerank'] = pd.Series(nx.pagerank(G))
df['Clustering'] = pd.Series(nx.clustering(G))
df['Closeness_Centrality'] = pd.Series(nx.closeness_centrality(G))

# Define target variable
df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

df.head()

,Department,Degree,Betweennes_Centrality,Degree_Centrality,Pagerank,Clustering,Closeness_Centrality,ManagementSalary
0,1,44,0.001124,0.043825,0.001224,0.276423,0.421991,0.0
1,1,52,0.001195,0.051793,0.001426,0.265306,0.422360,NaN
2,21,95,0.006570,0.094622,0.002605,0.297803,0.461490,NaN
3,21,71,0.001654,0.070717,0.001833,0.384910,0.441663,1.0
4,21,96,0.005547,0.095618,0.002526,0.318691,0.462152,1.0


In [7]:
features = ['Department', 'Degree', 'Betweennes_Centrality', 'Degree_Centrality', 'Pagerank'
            , 'Clustering', 'Closeness_Centrality']

features2 = ['Department', 'Degree', 'Betweennes_Centrality', 'Degree_Centrality', 'Pagerank'
            , 'Clustering', 'Closeness_Centrality', 'ManagementSalary']

features3 = ['Degree', 'Betweennes_Centrality', 'Degree_Centrality', 'Pagerank'
            , 'Clustering', 'Closeness_Centrality']



X_test = df.loc[df.ManagementSalary.isnull(), features3]
X_test

,Degree,Betweennes_Centrality,Degree_Centrality,Pagerank,Clustering,Closeness_Centrality
1,52,1.195088e-03,0.051793,0.001426,0.265306,0.422360
2,95,6.569720e-03,0.094622,0.002605,0.297803,0.461490
5,171,3.099469e-02,0.170319,0.004914,0.107002,0.501484
8,37,5.568287e-04,0.036853,0.001059,0.447059,0.413151
14,80,3.725905e-03,0.079681,0.002166,0.215784,0.442068
18,56,3.075746e-03,0.055777,0.001531,0.301188,0.442068
27,63,3.116899e-03,0.062749,0.001713,0.368852,0.430258
30,68,1.693505e-03,0.067729,0.001721,0.402797,0.443284
31,50,3.986823e-04,0.049801,0.001255,0.412234,0.418156
34,31,1.152684e-04,0.030876,0.000839,0.637931,0.382565


In [8]:


X_train = df.loc[~(df.ManagementSalary.isnull()), features3]
X_train

,Degree,Betweennes_Centrality,Degree_Centrality,Pagerank,Clustering,Closeness_Centrality
0,44,0.001124,0.043825,0.001224,0.276423,0.421991
3,71,0.001654,0.070717,0.001833,0.384910,0.441663
4,96,0.005547,0.095618,0.002526,0.318691,0.462152
6,115,0.012387,0.114542,0.003146,0.155183,0.475805
7,72,0.002818,0.071713,0.002002,0.287785,0.420156
9,40,0.000280,0.039841,0.001203,0.425320,0.356196
10,47,0.001084,0.046813,0.001303,0.315449,0.442472
11,76,0.003946,0.075697,0.002064,0.229174,0.431218
12,58,0.004715,0.057769,0.001715,0.302597,0.433151
13,180,0.023565,0.179283,0.004590,0.110900,0.503050


In [9]:
y_train = df.loc[~(df.ManagementSalary.isnull()), 'ManagementSalary']
y_train

0       0.0
3       1.0
4       1.0
6       1.0
7       0.0
9       0.0
10      0.0
11      0.0
12      1.0
13      1.0
15      0.0
16      0.0
17      0.0
19      0.0
20      0.0
21      1.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
28      1.0
29      1.0
32      0.0
33      0.0
35      1.0
36      1.0
38      0.0
39      0.0
41      0.0
       ... 
961     0.0
964     0.0
965     0.0
966     0.0
967     0.0
970     0.0
971     1.0
972     0.0
973     0.0
975     0.0
976     0.0
977     0.0
978     0.0
979     0.0
980     0.0
981     0.0
982     0.0
983     0.0
985     0.0
986     0.0
988     0.0
990     0.0
993     0.0
995     0.0
997     0.0
998     0.0
999     0.0
1002    0.0
1003    0.0
1004    0.0
Name: ManagementSalary, Length: 753, dtype: float64

In [10]:
clf = GradientBoostingClassifier(n_estimators=600, learning_rate=0.1, max_depth=5, random_state=0).fit(X_train, y_train)

outcome = pd.Series(clf.predict(X_test), index = X_test.index.values)

outcome

1       0.0
2       1.0
5       1.0
8       0.0
14      0.0
18      0.0
27      0.0
30      1.0
31      0.0
34      0.0
37      0.0
40      0.0
45      0.0
54      0.0
55      1.0
60      0.0
62      1.0
65      1.0
77      0.0
79      0.0
97      0.0
101     0.0
103     1.0
108     0.0
113     0.0
122     0.0
141     0.0
142     1.0
144     0.0
145     1.0
       ... 
913     0.0
914     0.0
915     0.0
918     0.0
923     0.0
926     0.0
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.0
950     0.0
951     0.0
953     0.0
959     0.0
962     0.0
963     0.0
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.0
Length: 252, dtype: float64

In [11]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

clf = MLPClassifier(hidden_layer_sizes = (10, 5), alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
clf.fit(X_train_scaled, y_train)
test_proba = clf.predict_proba(X_test_scaled)[:, 1]
outcome2 = pd.Series(test_proba, index = X_test.index.values)

outcome2

1       0.431170
2       0.967536
5       0.999294
8       0.354396
14      0.836707
18      0.637946
27      0.738537
30      0.788737
31      0.473422
34      0.323476
37      0.263709
40      0.590384
45      0.494283
54      0.608344
55      0.621192
60      0.526403
62      0.999465
65      0.977488
77      0.183787
79      0.386931
97      0.227662
101     0.153351
103     0.584856
108     0.380201
113     0.915519
122     0.112315
141     0.905397
142     0.996790
144     0.137697
145     0.870250
          ...   
913     0.107346
914     0.170042
915     0.002905
918     0.317705
923     0.056311
926     0.168415
931     0.113493
934     0.021777
939     0.016850
944     0.003316
945     0.061685
947     0.257721
950     0.326343
951     0.067272
953     0.057297
959     0.004209
962     0.003583
963     0.395360
968     0.161740
969     0.147525
974     0.152156
984     0.019288
987     0.190190
989     0.166242
991     0.170285
992     0.006209
994     0.004450
996     0.0034

In [13]:
def salary_predictions():
    
    # Your Code Here
    
    return # Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [14]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [15]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
# estimate node measurements prior making them features
soundarajan_hopcroft_cn = list(nx.cn_soundarajan_hopcroft(G, community = 'Department'))

In [18]:
soundarajan_hopcroft_ra = list(nx.ra_index_soundarajan_hopcroft(G, community='Department'))

In [19]:
resource_allocation = list(nx.resource_allocation_index(G))

In [20]:
jaccard_coef = list(nx.jaccard_coefficient(G))

In [21]:
adamic_adar = list(nx.adamic_adar_index(G))

In [22]:
# create dataframe
preferential_attachment = list(nx.preferential_attachment(G))
df1 = pd.DataFrame(index = [(x[0], x[1]) for x in preferential_attachment])
df1['preferential_attachment'] = [x[2] for x in preferential_attachment]


# add node measurements to the dataframe
df1['soundarajan_hopcroft_cn'] = [x[2] for x in soundarajan_hopcroft_cn]
df1['soundarajan_hopcroft_ra'] = [x[2] for x in soundarajan_hopcroft_ra]
df1['resource_allocation'] = [x[2] for x in resource_allocation]
df1['jaccard_coef'] = [x[2] for x in jaccard_coef]
df1['adamic_adar'] = [x[2] for x in adamic_adar]
df1['common_neighbours'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in df1.index.values]

df1

,preferential_attachment,soundarajan_hopcroft_cn,soundarajan_hopcroft_ra,resource_allocation,jaccard_coef,adamic_adar,common_neighbours
"(0, 2)",4180,6,0.0,0.055340,0.045802,1.257626,6
"(0, 3)",3124,3,0.0,0.021388,0.027273,0.605728,3
"(0, 4)",4224,3,0.0,0.021388,0.022222,0.605728,3
"(0, 7)",3168,4,0.0,0.061668,0.036364,0.938816,4
"(0, 8)",1628,1,0.0,0.011628,0.012821,0.224500,1
"(0, 9)",1760,2,0.0,0.041931,0.025000,0.510499,2
"(0, 10)",2068,1,0.0,0.005882,0.011236,0.194712,1
"(0, 11)",3344,4,0.0,0.053661,0.035088,0.899701,4
"(0, 12)",2552,4,0.0,0.065762,0.041667,0.953236,4
"(0, 13)",7920,7,0.0,0.199248,0.032558,1.943251,7


In [23]:
# merge future connections
df_merged = df1.merge(future_connections, left_index = True, right_index =  True)
df_merged

,preferential_attachment,soundarajan_hopcroft_cn,soundarajan_hopcroft_ra,resource_allocation,jaccard_coef,adamic_adar,common_neighbours,Future Connection
"(0, 2)",4180,6,0.0,0.055340,0.045802,1.257626,6,0.0
"(0, 3)",3124,3,0.0,0.021388,0.027273,0.605728,3,0.0
"(0, 4)",4224,3,0.0,0.021388,0.022222,0.605728,3,0.0
"(0, 7)",3168,4,0.0,0.061668,0.036364,0.938816,4,0.0
"(0, 8)",1628,1,0.0,0.011628,0.012821,0.224500,1,0.0
"(0, 9)",1760,2,0.0,0.041931,0.025000,0.510499,2,NaN
"(0, 10)",2068,1,0.0,0.005882,0.011236,0.194712,1,0.0
"(0, 11)",3344,4,0.0,0.053661,0.035088,0.899701,4,0.0
"(0, 12)",2552,4,0.0,0.065762,0.041667,0.953236,4,0.0
"(0, 13)",7920,7,0.0,0.199248,0.032558,1.943251,7,0.0


In [24]:
df_merged.shape

(488446, 8)

In [25]:
features = ['preferential_attachment', 'soundarajan_hopcroft_cn', 'soundarajan_hopcroft_ra', 'resource_allocation'
           , 'jaccard_coef', 'adamic_adar', 'common_neighbours']

features2 = ['preferential_attachment', 'soundarajan_hopcroft_cn', 'soundarajan_hopcroft_ra', 'resource_allocation'
           , 'jaccard_coef', 'adamic_adar', 'common_neighbours', 'Future Connection']



In [26]:
X_train = df_merged.loc[~(df_merged['Future Connection'].isnull()), features]
X_train

,preferential_attachment,soundarajan_hopcroft_cn,soundarajan_hopcroft_ra,resource_allocation,jaccard_coef,adamic_adar,common_neighbours
"(0, 2)",4180,6,0.0,0.055340,0.045802,1.257626,6
"(0, 3)",3124,3,0.0,0.021388,0.027273,0.605728,3
"(0, 4)",4224,3,0.0,0.021388,0.022222,0.605728,3
"(0, 7)",3168,4,0.0,0.061668,0.036364,0.938816,4
"(0, 8)",1628,1,0.0,0.011628,0.012821,0.224500,1
"(0, 10)",2068,1,0.0,0.005882,0.011236,0.194712,1
"(0, 11)",3344,4,0.0,0.053661,0.035088,0.899701,4
"(0, 12)",2552,4,0.0,0.065762,0.041667,0.953236,4
"(0, 13)",7920,7,0.0,0.199248,0.032558,1.943251,7
"(0, 14)",3520,11,0.0,0.177760,0.099099,2.635144,11


In [27]:
y_train = df_merged.loc[~(df_merged['Future Connection'].isnull()), 'Future Connection']
y_train

(0, 2)          0.0
(0, 3)          0.0
(0, 4)          0.0
(0, 7)          0.0
(0, 8)          0.0
(0, 10)         0.0
(0, 11)         0.0
(0, 12)         0.0
(0, 13)         0.0
(0, 14)         1.0
(0, 15)         0.0
(0, 16)         0.0
(0, 21)         0.0
(0, 22)         0.0
(0, 23)         0.0
(0, 24)         0.0
(0, 25)         0.0
(0, 26)         0.0
(0, 27)         0.0
(0, 28)         0.0
(0, 29)         0.0
(0, 30)         0.0
(0, 31)         0.0
(0, 32)         0.0
(0, 33)         0.0
(0, 34)         0.0
(0, 36)         0.0
(0, 37)         0.0
(0, 39)         0.0
(0, 40)         0.0
               ... 
(995, 1003)     0.0
(995, 1004)     0.0
(996, 998)      0.0
(996, 999)      0.0
(996, 1000)     0.0
(996, 1001)     0.0
(996, 1002)     0.0
(996, 1003)     0.0
(996, 1004)     0.0
(997, 999)      0.0
(997, 1000)     0.0
(997, 1001)     0.0
(997, 1002)     0.0
(997, 1003)     0.0
(998, 1000)     0.0
(998, 1001)     0.0
(998, 1002)     0.0
(998, 1003)     0.0
(998, 1004)     0.0


In [ ]:
X_test = df_merged.loc[df_merged['Future Connection'].isnull(), features]
X_test

,preferential_attachment,soundarajan_hopcroft_cn,soundarajan_hopcroft_ra,resource_allocation,jaccard_coef,adamic_adar,common_neighbours
"(0, 9)",1760,2,0.0,0.041931,0.025000,0.510499,2
"(0, 19)",3168,4,0.0,0.064557,0.036364,0.952749,4
"(0, 20)",3256,7,0.0,0.090283,0.064220,1.597693,7
"(0, 35)",2596,1,0.0,0.005848,0.010000,0.194490,1
"(0, 38)",2068,0,0.0,0.000000,0.000000,0.000000,0
"(0, 42)",3300,11,0.0,0.133897,0.103774,2.464933,11
"(0, 43)",616,1,0.0,0.011628,0.018182,0.224500,1
"(0, 50)",748,0,0.0,0.000000,0.000000,0.000000,0
"(0, 53)",1760,8,0.0,0.132775,0.106667,1.928698,8
"(0, 54)",2244,3,0.0,0.044648,0.033333,0.689945,3


In [ ]:
clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, max_depth=10, random_state=0).fit(X_train, y_train)

clf

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=150, presort='auto', random_state=0,
              subsample=1.0, verbose=0, warm_start=False)

In [ ]:
outcome_grad = pd.Series(clf.predict(X_test), index = X_test.index.values)
outcome_grad

(0, 9)          0.0
(0, 19)         0.0
(0, 20)         0.0
(0, 35)         0.0
(0, 38)         0.0
(0, 42)         1.0
(0, 43)         0.0
(0, 50)         0.0
(0, 53)         1.0
(0, 54)         0.0
(0, 62)         0.0
(0, 63)         0.0
(0, 69)         0.0
(0, 72)         0.0
(0, 83)         0.0
(0, 90)         0.0
(0, 91)         0.0
(0, 95)         0.0
(0, 100)        0.0
(0, 106)        1.0
(0, 108)        0.0
(0, 109)        0.0
(0, 110)        0.0
(0, 118)        0.0
(0, 122)        0.0
(0, 131)        0.0
(0, 133)        1.0
(0, 140)        0.0
(0, 149)        0.0
(0, 151)        0.0
               ... 
(988, 989)      0.0
(988, 996)      0.0
(988, 997)      0.0
(988, 1000)     0.0
(988, 1002)     0.0
(989, 994)      0.0
(989, 996)      0.0
(989, 1003)     0.0
(989, 1004)     0.0
(990, 994)      0.0
(990, 998)      0.0
(991, 994)      0.0
(991, 999)      0.0
(991, 1003)     0.0
(992, 994)      0.0
(992, 995)      0.0
(992, 997)      0.0
(992, 1000)     0.0
(993, 1000)     0.0


In [31]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

In [32]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


In [34]:
clf = MLPClassifier(hidden_layer_sizes = (10, 5), alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
clf

MLPClassifier(activation='relu', alpha=5, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=0,
       warm_start=False)

In [35]:
clf.fit(X_train_scaled, y_train)
test_proba = clf.predict_proba(X_test_scaled)[:, 1]


In [36]:
outcome3 = pd.Series(test_proba, index = X_test.index.values)

outcome3

(0, 9)          0.032650
(0, 19)         0.119786
(0, 20)         0.587989
(0, 35)         0.013181
(0, 38)         0.013181
(0, 42)         0.970654
(0, 43)         0.016780
(0, 50)         0.013181
(0, 53)         0.823873
(0, 54)         0.064727
(0, 62)         0.855008
(0, 63)         0.090275
(0, 69)         0.027535
(0, 72)         0.013181
(0, 83)         0.480254
(0, 90)         0.034209
(0, 91)         0.015049
(0, 95)         0.261044
(0, 100)        0.032041
(0, 106)        0.997620
(0, 108)        0.014173
(0, 109)        0.013181
(0, 110)        0.013181
(0, 118)        0.013181
(0, 122)        0.034996
(0, 131)        0.061189
(0, 133)        0.960492
(0, 140)        0.138728
(0, 149)        0.300352
(0, 151)        0.013181
                  ...   
(988, 989)      0.013181
(988, 996)      0.013181
(988, 997)      0.090535
(988, 1000)     0.013181
(988, 1002)     0.013181
(989, 994)      0.013181
(989, 996)      0.013181
(989, 1003)     0.013181
(989, 1004)     0.013181


In [39]:
goal = future_connections[pd.isnull(future_connections['Future Connection'])]
goal = goal.copy()

In [40]:
goal['probability'] = [outcome3[x] for x in goal.index]
goal['probability'] 

(107, 348)    0.031115
(542, 751)    0.013181
(20, 426)     0.917823
(50, 989)     0.013181
(942, 986)    0.013181
(324, 857)    0.013181
(13, 710)     0.353751
(19, 271)     0.327381
(319, 878)    0.013181
(659, 707)    0.013181
(49, 843)     0.013181
(208, 893)    0.013181
(377, 469)    0.013181
(405, 999)    0.016339
(129, 740)    0.015019
(292, 618)    0.017243
(239, 689)    0.013181
(359, 373)    0.013181
(53, 523)     0.247920
(276, 984)    0.013181
(202, 997)    0.013181
(604, 619)    0.083844
(270, 911)    0.013181
(261, 481)    0.127908
(200, 450)    1.000000
(213, 634)    0.013181
(644, 735)    0.061151
(346, 553)    0.013181
(521, 738)    0.013181
(422, 953)    0.016467
                ...   
(672, 848)    0.013181
(28, 127)     0.996589
(202, 661)    0.013181
(54, 195)     0.999999
(295, 864)    0.013181
(814, 936)    0.013181
(839, 874)    0.013181
(139, 843)    0.013181
(461, 544)    0.013181
(68, 487)     0.013181
(622, 932)    0.013181
(504, 936)    0.015293
(479, 528) 